In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Загрузка данных
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
raw_data = pd.read_csv(url)

# Преобразование данных в ежемесячные средние температуры
raw_data['Date'] = pd.to_datetime(raw_data['Date'])
monthly_data = raw_data.resample('M', on='Date').mean()

# Разложение временного ряда
decomposition = seasonal_decompose(raw_data['Temp'], period=12)

trend = decomposition.trend
seasonality = decomposition.seasonal
residual = decomposition.resid

plt.figure(figsize=(10,8))
plt.subplot(411)
plt.plot(raw_data['Temp'], label='Original', color='blue')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend', color='blue')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonality,label='Seasonality', color='blue')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals', color='blue')
plt.legend(loc='best')
plt.tight_layout()

plt.show()

# Разделение данных на обучающую и тестовую выборки
train = monthly_data.iloc[:-12]
test = monthly_data.iloc[-12:]

# Применение метода Хольта-Винтерса
model = ExponentialSmoothing(train, seasonal="additive", seasonal_periods=12)
fitted_model = model.fit()

# Прогнозирование
forecast = fitted_model.forecast(steps=12)

# Визуализация результатов
plt.plot(train.index, train, label="Обучающая выборка")
plt.plot(test.index, test, label="Тестовая выборка")
plt.plot(test.index, forecast, label="Прогноз")
plt.xlabel("Год")
plt.ylabel("Средняя температура (°C)")
plt.title("Прогноз средней температуры с помощью метода Хольта-Уинтерса")
plt.legend()
plt.show()

# Оценка прогноза с использованием различных метрик
mae = mean_absolute_error(test, forecast)
mse = mean_squared_error(test, forecast)
r2 = r2_score(test, forecast)

print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Коэффициент детерминации (R2): {r2:.2f}")